## <center>Initial Processing & Feature Engineering</center>

 The steps involved in this process were,
 1. [Reading Historing Loan File](#read)
 2. [Feature Processing](#feature_pro)
 3. [Text & Sentiment Analysis](#text)
 4. [Imputing Zip Code with Median Income & Population](#zip)

In [1]:
import pandas as pd
import numpy as np

<a id='read'></a>
#### Reading Historing Loan File

In [2]:
#reading data
data = pd.read_csv('data/LoanStats3a_securev1.csv')

/Users/ajaykliyara/anaconda_py3/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.shape

(42540, 115)

In [4]:
data.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


<a id='feature_pro'></a>
#### Feature Processing

In [5]:
for i in data.columns:
    print(i)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_il_6m
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
tot

In [6]:
#this verifies number of id's equal to number of records, therefore every row represents a
#unique customer
print(len(data['id'].unique()))
print(len(data['member_id'].unique()))


42537
42536


In [7]:
#loan_amnt != funded_amnt, therefore removing funded amnt
np.mean(data['loan_amnt'] != data['funded_amnt'])

0.047061589092618712

In [8]:
#emp title stands for designation of applier, high dimensional, leaving out for now
data['emp_title'].unique().shape

(30660,)

In [9]:
data['term'].unique()

array([' 36 months', ' 60 months', nan], dtype=object)

In [10]:
data.groupby('loan_status')['loan_status'].count()

loan_status
Charged Off                                             5655
Current                                                  445
Default                                                    1
Does not meet the credit policy. Status:Charged Off      761
Does not meet the credit policy. Status:Fully Paid      1988
Fully Paid                                             33663
In Grace Period                                            9
Late (16-30 days)                                          3
Late (31-120 days)                                        10
Name: loan_status, dtype: int64

In [11]:
#all applications are individual , no joints therefore will drop all joint attributes.
data['application_type'].unique()

array(['INDIVIDUAL', nan], dtype=object)

In [12]:
data['mort_acc'].unique()

array([ nan])

In [13]:
#id,member_id - unique customer id's , does not contribute to ML models
field_deleted = ['id','member_id','issue_d','pymnt_plan','url','title','addr_state',
                 'initial_list_status','out_prncp','out_prncp_inv','total_pymnt',
                 'total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee',
                 'recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d',
                 'last_credit_pull_d','collections_12_mths_ex_med','policy_code','annual_inc_joint','dti_joint','verification_status_joint','open_il_6m','open_il_12m','open_il_24m','total_bal_il','open_rv_12m','open_rv_24m','total_cu_tl','mort_acc','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc',
                 'mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']
field_retained = ['loan_amnt','funded_amnt','funded_amnt_inv','zip_code','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc','verification_status','desc','purpose','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths',
                  'mths_since_last_delinq','mths_since_last_record','open_acc','total_acc','revol_bal','revol_util','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt','tot_cur_bal','mths_since_rcnt_il','il_util','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
                  'bc_util','chargeoff_within_12_mths','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','fico_range_high','fico_range_low','loan_status']

In [14]:
#data to be retained
data_ss = data.loc[:,field_retained]

In [15]:
data_ss.shape

(42540, 52)

In [16]:
len(data_ss.zip_code.unique())

838

In [17]:
data_ss.to_csv('data/loan_ss_2007_2011.csv',index=False)

In [18]:
data_ss_r = pd.read_csv('data/loan_ss_2007_2011.csv')

In [19]:
data_ss_r.shape

(42540, 52)

In [20]:
data_ss_r.head()

,loan_amnt,funded_amnt,funded_amnt_inv,zip_code,term,int_rate,installment,grade,sub_grade,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,fico_range_high,fico_range_low,loan_status
0,5000.0,5000.0,4975.0,860xx,36 months,10.65%,162.87,B,B2,10+ years,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,739.0,735.0,Fully Paid
1,2500.0,2500.0,2500.0,309xx,60 months,15.27%,59.83,C,C4,< 1 year,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,744.0,740.0,Charged Off
2,2400.0,2400.0,2400.0,606xx,36 months,15.96%,84.33,C,C5,10+ years,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,739.0,735.0,Fully Paid
3,10000.0,10000.0,10000.0,917xx,36 months,13.49%,339.31,C,C1,10+ years,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,694.0,690.0,Fully Paid
4,3000.0,3000.0,3000.0,972xx,60 months,12.69%,67.79,B,B5,1 year,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,699.0,695.0,Current


In [21]:
#retained columns
data_ss_r.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'zip_code', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'desc',
       'purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'total_acc', 'revol_bal', 'revol_util', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'mths_since_rcnt_il',
       'il_util', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'fico_range_high', 'fico_range_low',
       'loan_status'],
      dtype='object')

In [22]:
data_ss_r.groupby('loan_status')['loan_status'].count()

loan_status
Charged Off                                             5655
Current                                                  445
Default                                                    1
Does not meet the credit policy. Status:Charged Off      761
Does not meet the credit policy. Status:Fully Paid      1988
Fully Paid                                             33663
In Grace Period                                            9
Late (16-30 days)                                          3
Late (31-120 days)                                        10
Name: loan_status, dtype: int64

In [23]:
data_ss_r2 = data_ss_r.loc[data_ss_r.loan_status.isin(['Charged Off','Fully Paid']),:]

In [24]:
data_ss_r2.groupby('loan_status')['loan_status'].count()

loan_status
Charged Off     5655
Fully Paid     33663
Name: loan_status, dtype: int64

In [25]:
data_ss_r2.to_csv('data/loan_ss_2007_2011.csv',index=False)

In [26]:
data_ss_r2.shape

(39318, 52)

In [27]:
data = pd.read_csv('data/loan_ss_2007_2011.csv')

<a id='text'></a>
#### Text Analysis

In [28]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_rmStop(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        n_token = re.sub("[^\w]", "",  token)
        if len(n_token) != 0:
            filtered_tokens.append(n_token)
    #removing stop words
    texts = [word for word in filtered_tokens if word not in stopwords]
    
    return texts

In [29]:
def tokenize_and_rmStop_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        n_token = re.sub("[^\w]", "",  token)
        if len(n_token) != 0:
            filtered_tokens.append(n_token)
    #removing stop words
    texts = [word for word in filtered_tokens if word not in stopwords]
    
    #stemming remaining words
    stems = [stemmer.stem(t) for t in texts]
    
    return stems

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
def tfidf(terms, in_max_df = 0.7, in_max_features = 200000, 
          in_min_df = 0.1, in_use_idf = True, 
          in_tokenizer = tokenize_and_rmStop_and_stem,
          in_ngram_range = (1, 1)):
    # compute the TFIDF model
    tfidf_vectorizer = TfidfVectorizer(max_df = in_max_df, max_features = in_max_features,
                                       min_df = in_min_df, 
                                       use_idf = in_use_idf, 
                                       tokenizer = in_tokenizer, 
                                       ngram_range = in_ngram_range)
    # compute the TFIDF values
    tfidf_matrix = tfidf_vectorizer.fit_transform(terms) #fit the vectorizer to synopses
    
    # convert to a matrix and get the term names
    tfidf_matrix = tfidf_matrix.A # to array
    term_names = tfidf_vectorizer.get_feature_names()
    
    # create a pandas dataframe from the matrix
    df = pd.DataFrame()
    df = pd.DataFrame(data = tfidf_matrix, columns = term_names)
    
    # return the matrix and data frame to the caller
    return tfidf_matrix, df

In [31]:
matrix,df = tfidf(data.desc.values.astype(str))

In [32]:
df.head()

,ad,bill,borrow,br,card,consolid,credit,current,debt,fund,...,pay,payment,plan,rate,thank,time,use,work,would,year
0,0.382020,0.0,0.391159,0.388257,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.154319,0.0,0.158010,0.235257,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.606425,0.0,0.0,0.000000,0.248817,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.176156,0.0,0.360741,0.179032,0.0,0.0,0.0,0.0,0.0,0.0,...,0.446999,0.0,0.000000,0.0,0.0,0.307005,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [33]:
df.columns

Index(['ad', 'bill', 'borrow', 'br', 'card', 'consolid', 'credit', 'current',
       'debt', 'fund', 'get', 'help', 'interest', 'job', 'like', 'loan',
       'make', 'money', 'month', 'nan', 'need', 'one', 'pay', 'payment',
       'plan', 'rate', 'thank', 'time', 'use', 'work', 'would', 'year'],
      dtype='object')

In [34]:
#higher borrow indicates multiple comments
#dropping columns by intution
df = df.drop(['ad','br','nan','would','use','like','get','use','borrow','make','need','one'],axis=1)

In [35]:
df.head()

,bill,card,consolid,credit,current,debt,fund,help,interest,job,...,money,month,pay,payment,plan,rate,thank,time,work,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.613525,0.0,0.000000,0.0,0.606425,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.446999,0.0,0.000000,0.0,0.0,0.307005,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [36]:
from afinn import Afinn

In [37]:
afinn = Afinn()

In [38]:
txt_arr = data.desc.values.astype(str)

In [39]:
txt_senti_score = [afinn.score(i) for i in txt_arr]

In [40]:
#assigning sentiment score
data['desc_senti_score'] = txt_senti_score

In [41]:
final_df = pd.concat([data,df],axis=1)

In [42]:
final_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,zip_code,term,int_rate,installment,grade,sub_grade,emp_length,...,money,month,pay,payment,plan,rate,thank,time,work,year
0,5000.0,5000.0,4975.0,860xx,36 months,10.65%,162.87,B,B2,10+ years,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,2500.0,2500.0,2500.0,309xx,60 months,15.27%,59.83,C,C4,< 1 year,...,0.613525,0.0,0.000000,0.0,0.606425,0.0,0.0,0.000000,0.0,0.0
2,2400.0,2400.0,2400.0,606xx,36 months,15.96%,84.33,C,C5,10+ years,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,10000.0,10000.0,10000.0,917xx,36 months,13.49%,339.31,C,C1,10+ years,...,0.000000,0.0,0.446999,0.0,0.000000,0.0,0.0,0.307005,0.0,0.0
4,5000.0,5000.0,5000.0,852xx,36 months,7.90%,156.46,A,A4,3 years,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


<a id='zip'></a>
#### Imputing Zip Code with Median Income & Population

In [43]:
#reading zipcode data

zip_code_df = pd.read_excel('data/zip_attr.xlsx')

In [44]:
zip_code_df.head(2)

,Zip,Median,Mean,Pop
0,1001,56662.5735,66687.8,16445
1,1002,49853.4177,75062.6,28069


In [45]:
np.unique(zip_code_df.Zip.apply(str).apply(len))

array([4, 5])

In [46]:
def exp_zip(zip):
    if len(str(zip)) < 5:
        return '0' + str(zip)
    else:
        return str(zip)

In [47]:
zip_code_df['zip_5'] = [exp_zip(zip) for zip in zip_code_df.Zip.values]

In [48]:
zip_code_df.head(2)

,Zip,Median,Mean,Pop,zip_5
0,1001,56662.5735,66687.8,16445,01001
1,1002,49853.4177,75062.6,28069,01002


In [49]:
zip_code_df['zip_3'] = [zip[:3] for zip in zip_code_df.zip_5.values]

In [50]:
zip_code_df.head(2)

,Zip,Median,Mean,Pop,zip_5,zip_3
0,1001,56662.5735,66687.8,16445,01001,010
1,1002,49853.4177,75062.6,28069,01002,010


In [51]:
zip_code_agg = zip_code_df.groupby('zip_3')['Median','Pop'].mean()

In [52]:
zip_code_agg = zip_code_agg.reset_index()
zip_code_agg.head(2)

,zip_3,Median,Pop
0,010,61658.753829,7257.177419
1,011,43307.118338,12896.461538


In [53]:
np.unique(zip_code_agg.Pop.isnull())

array([False], dtype=bool)

In [54]:
zip_code_agg.to_csv('data/zip3_agg.csv',index=False)

In [55]:
#join with final df
final_df['zip_3'] = [zip[:3] for zip in final_df.zip_code.values]

In [56]:
final_df.shape

(39318, 75)

In [57]:
final_df_cols = list(final_df.columns)

In [58]:
join_df = pd.merge(final_df,zip_code_agg,how='inner',on='zip_3')

In [59]:
join_df.shape

(39301, 77)

In [60]:
join_df = join_df.drop(['zip_code','zip_3'],axis=1)
join_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'desc', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'total_acc', 'revol_bal', 'revol_util', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'mths_since_rcnt_il',
       'il_util', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'fico_range_high', 'fico_range_low',
       'loan_status', 'desc_senti_score', 'bill', 'card', 'consolid', 'c

In [61]:
join_df = join_df.rename(columns={'Median':'zip_median_income','Pop':'zip_pop'})

In [62]:
join_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'desc', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'total_acc', 'revol_bal', 'revol_util', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'mths_since_rcnt_il',
       'il_util', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'fico_range_high', 'fico_range_low',
       'loan_status', 'desc_senti_score', 'bill', 'card', 'consolid', 'c

In [63]:
final_df = join_df.copy()

In [64]:
final_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'desc', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'total_acc', 'revol_bal', 'revol_util', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'mths_since_rcnt_il',
       'il_util', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'fico_range_high', 'fico_range_low',
       'loan_status', 'desc_senti_score', 'bill', 'card', 'consolid', 'c

In [65]:
final_df.to_csv('data/loan_with_text_raw.csv',index=False)